In [1]:
import openai
import pandas as pd
from dotenv import load_dotenv
import os

# .env 파일에서 OpenAI API 키 로드
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
# CSV 파일 읽기
csv_file_path = 'qa_results.csv'
df = pd.read_csv(csv_file_path)

# 평가 함수 정의
def evaluate_conversation(question, gpt4o_answer, gemma2_fine_answer):
    messages = [
        {"role": "system", "content": "당신은 대화 평가 전문가입니다."},
        {"role": "user", "content": f"""
        주어진 질문과 두 대답(GPT-4.0 및 Gemma2.0)을 평가해주세요. 평가 기준은 정확성, 관련성, 완전성, 명확성 네 가지입니다.

        질문: "{question}"

        GPT-4.0의 답변: "{gpt4o_answer}"
        Gemma2.0의 답변: "{gemma2_fine_answer}"

        각 평가 기준에 대해 점수를 매겨주세요 (1-10):
        1. 정확성 (Accuracy): 
        2. 관련성 (Relevance): 
        3. 완전성 (Completeness): 
        4. 명확성 (Clarity): 
        """}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150
    )

    return response.choices[0].message['content'].strip()

# 대화 평가
results = []
for index, row in df.iterrows():
    question = row['question']
    gpt4o_answer = row['gpt4oAnswer']
    gemma2_fine_answer = row['gemma2FineAnswer']
    
    evaluation = evaluate_conversation(question, gpt4o_answer, gemma2_fine_answer)
    results.append(evaluation)

# 평가 결과 저장
df['evaluation'] = results
df.to_csv('qa_evaluation_results.csv', index=False)

print("평가 완료. 평가 결과는 'qa_evaluation_results.csv' 파일에 저장되었습니다.")

평가 완료. 평가 결과는 'qa_evaluation_results.csv' 파일에 저장되었습니다.
